In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("csv_review_nlv.csv")

In [7]:
data.head()

,text,date,cool,funny,stars,business_id,useful,user_id,review_id
0,Had the Tonkotsu ramen for lunch upon the reco...,2017-05-02 20:37:33 UTC,0,0,4,-DnaKAs2oK3rXfrjSvn9ew,0,70p7ZC7JNkAn4YHJL2oLqQ,SvrM4R0Sn1085JldYArCwA
1,"Service was quick, food was excellent am glad ...",2015-10-09 01:30:39 UTC,0,0,5,-DnaKAs2oK3rXfrjSvn9ew,0,vRhxHrWwHpPDbzzKsCKIQw,6p-MEBAuZ4dhOA1k24vgdw
2,"I actually think that the place is a 3.5, so, ...",2015-06-07 00:33:50 UTC,0,0,4,-DnaKAs2oK3rXfrjSvn9ew,0,Bxf1jhd66YvcI0GFTIh3yQ,5P--fn1aEuuzsRiIgRcfCA
3,I eat alot of ramen and alot of curry.. but i ...,2017-07-22 20:35:05 UTC,0,0,5,-DnaKAs2oK3rXfrjSvn9ew,0,eRwwZ7CjgDuywdNZ_mtgKQ,XJLHswUbJYbzj9CmHpIsSw
4,Stopped in on a whim with my husband. Our stom...,2016-01-03 02:22:19 UTC,0,0,5,-DnaKAs2oK3rXfrjSvn9ew,0,PBSkpQxdUzx-4LyvEgX1KQ,FieX2hOMO2R0z5hj59sXCg


In [14]:
from gensim import corpora, models
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\7353228\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [17]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import en_core_web_sm

C:\Users\7353228\Anaconda\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """


In [31]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [19]:
data.head()

,text,date,cool,funny,stars,business_id,useful,user_id,review_id
0,Had the Tonkotsu ramen for lunch upon the reco...,2017-05-02 20:37:33 UTC,0,0,4,-DnaKAs2oK3rXfrjSvn9ew,0,70p7ZC7JNkAn4YHJL2oLqQ,SvrM4R0Sn1085JldYArCwA
1,"Service was quick, food was excellent am glad ...",2015-10-09 01:30:39 UTC,0,0,5,-DnaKAs2oK3rXfrjSvn9ew,0,vRhxHrWwHpPDbzzKsCKIQw,6p-MEBAuZ4dhOA1k24vgdw
2,"I actually think that the place is a 3.5, so, ...",2015-06-07 00:33:50 UTC,0,0,4,-DnaKAs2oK3rXfrjSvn9ew,0,Bxf1jhd66YvcI0GFTIh3yQ,5P--fn1aEuuzsRiIgRcfCA
3,I eat alot of ramen and alot of curry.. but i ...,2017-07-22 20:35:05 UTC,0,0,5,-DnaKAs2oK3rXfrjSvn9ew,0,eRwwZ7CjgDuywdNZ_mtgKQ,XJLHswUbJYbzj9CmHpIsSw
4,Stopped in on a whim with my husband. Our stom...,2016-01-03 02:22:19 UTC,0,0,5,-DnaKAs2oK3rXfrjSvn9ew,0,PBSkpQxdUzx-4LyvEgX1KQ,FieX2hOMO2R0z5hj59sXCg


In [21]:
reviews = data.text.values.tolist()

In [22]:
# Remove new line characters
reviews = [re.sub('\s+', ' ', sent) for sent in reviews]

In [23]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(reviews))

In [32]:
data_words[1]

['service',
 'was',
 'quick',
 'food',
 'was',
 'excellent',
 'am',
 'glad',
 'my',
 'wife',
 'and',
 'daughter',
 'saw',
 'this',
 'place',
 'and',
 'read',
 'the',
 'reviews',
 'before',
 'we',
 'went',
 'to',
 'the',
 'cannery',
 'buffet']

In [25]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['had', 'the', 'tonkotsu_ramen', 'for', 'lunch', 'upon', 'the', 'recommendation', 'from', 'co_worker', 'very', 'well', 'made', 'with', 'tasty', 'broth', 'the', 'slices', 'of', 'pork', 'just', 'melted', 'in', 'my', 'mouth', 'very', 'filling', 'and', 'tasty', 'have', 'returned', 'to', 'try', 'out', 'some', 'of', 'the', 'other', 'soups', 'update', 'the', 'vegetable', 'udon', 'noodle_soup', 'was', 'also', 'very', 'good', 'the', 'ambience', 'and', 'decor', 'need', 'some', 'work', 'but', 'the', 'service', 'is', 'very', 'courteous', 'and', 'the', 'soups', 'are', 'great']


In [26]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#python3 -m spacy download en
nlp = en_core_web_sm.load()

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])